# Cálculos de Ganho PIBIC

### Python reading file setup

In [424]:
import pandas as pd

file_path = "joined_file.tsv"
header = ['time', 'x', 'y', 'z']
df = pd.read_csv(file_path, sep='\t', decimal=',') # Valores em g
df.columns = header
df.head()

,time,x,y,z
0,0.000000,0.001585,0.001060,0.001670
1,0.000586,0.002384,0.002132,0.002123
2,0.001172,0.002092,0.002555,0.002099
3,0.001758,0.002629,0.003864,0.002976
4,0.002344,0.002211,0.004657,0.003649


In [425]:
df[['x', 'y', 'z']] *= 100 # Transformando em mV
df.head()

,time,x,y,z
0,0.000000,0.1585,0.1060,0.1670
1,0.000586,0.2384,0.2132,0.2123
2,0.001172,0.2092,0.2555,0.2099
3,0.001758,0.2629,0.3864,0.2976
4,0.002344,0.2211,0.4657,0.3649


### Attenuations

In [426]:
from enum import Enum
class Attenuation(Enum):
    ATTENUATION_0DB = {'full-range': (0, 1100), 'linear-range': (100, 950)}
    ATTENUATION_2_5DB = {'full-range': (0, 1500), 'linear-range': (100, 1250)}
    ATTENUATION_6DB = {'full-range': (0, 2200), 'linear-range': (150, 1750)}
    ATTENUATION_11DB = {'full-range': (0, 3300), 'linear-range': (150, 2450)}


In [476]:
ATTENUATION = Attenuation.ATTENUATION_11DB.value
ATTENUATION

{'full-range': (0, 3300), 'linear-range': (150, 2450)}

##  Cálculo Resolução

In [477]:
MAX_VOLTAGE = ATTENUATION['full-range'][1]
LINEAR_MAX = ATTENUATION['linear-range'][1]
LINEAR_MIN = ATTENUATION['linear-range'][0]


RESOLUCAO_ESP = MAX_VOLTAGE/4096 # Resolução de 12 bits
LINEAR_RANGE = LINEAR_MAX - LINEAR_MIN # Região linear para 1.1V
RESOLUCAO_ESP # Em mV

0.8056640625

## Cálculo do ganho

In [478]:
no_null_df = df[(df['x'] != 0) & (df['y'] != 0) & (df['z'] != 0)].abs() # Criando um DataFrame sem valores nulos para calcular a resolução mínima
no_null_df['x'].nsmallest(100)

4229     0.0001
6047     0.0001
8458     0.0001
13528    0.0001
15372    0.0001
          ...  
19871    0.0005
21582    0.0005
27105    0.0005
47527    0.0005
52914    0.0005
Name: x, Length: 100, dtype: float64

In [479]:
abs_min_values = no_null_df.min().to_dict()
abs_min_values # Em mV

{'time': 0.0,
 'x': 9.999999999999999e-05,
 'y': 0.0006000000000000001,
 'z': 0.0006000000000000001}

In [480]:
min_gain = RESOLUCAO_ESP/abs_min_values['x']
min_gain # Ganho mínimo necessário

8056.640625000001

In [481]:
abs_max_values = no_null_df.max().to_dict()
abs_max_values

{'time': 686.432785, 'x': 102.6948, 'y': 112.714, 'z': 64.4559}

In [482]:
mean = (LINEAR_MAX + LINEAR_MIN)/2
df_plus_mean = df.copy(deep=True)
df_plus_mean[['x', 'y', 'z']] = df_plus_mean[['x', 'y', 'z']] + mean
max_values = df_plus_mean.max().to_dict()
max_values

{'time': 686.432785, 'x': 1388.615, 'y': 1405.6069, 'z': 1363.9586}

In [483]:
max_value = max_values['y']
max_gain = LINEAR_MAX/max_value
max_gain

1.743019332076415

In [484]:
int(max_value * max_gain)

2450

In [485]:
gain = max_gain
#gain = 100

In [486]:
df_gt_resol = (df[['x', 'y', 'z']] * gain).abs() # DataFrame dos valores absolutos vezes o ganho máximo
df_gt_resol = df_gt_resol[
    (((df_gt_resol['x'] <= MAX_VOLTAGE) & (df_gt_resol['x'] >= RESOLUCAO_ESP)) | (df_gt_resol['x'] == 0)) & 
    (((df_gt_resol['y'] <= MAX_VOLTAGE) & (df_gt_resol['y'] >= RESOLUCAO_ESP)) | (df_gt_resol['y'] == 0)) & 
    (((df_gt_resol['z'] <= MAX_VOLTAGE) & (df_gt_resol['z'] >= RESOLUCAO_ESP)) | (df_gt_resol['z'] == 0))]
data_loss_percentage = 100 - len(df_gt_resol)/len(df) * 100
data_loss_percentage

23.883527987466167

Para Atenuação de 0dB: 
    - Ganho máximo de 1.506485260468923
    - Perda de 9.991365190143853% dos dados

Para Atenuação de 2.5dB:
    - Ganho máximo de 1.6013181538620784
    - Perda de 12.635664435265639% dos dados

Para Atenuação de 6dB:
    - Ganho máximo de 1.657814097274279
    - Perda de 17.43127759578408% dos dados

Para Atenuação de 11dB:
    - Ganho máximo de 1.743019332076415
    - Perda de 23.883527987466167% dos dados

In [487]:
def best_performance(df: pd.DataFrame):
    best_percentage = 110
    best_gain = -1
    for gain in range(1, 100):
        df_gt_resol = (df[['x', 'y', 'z']] * gain).abs() # DataFrame dos valores absolutos vezes o ganho máximo
        df_gt_resol = df_gt_resol[
            (((df_gt_resol['x'] <= LINEAR_MAX) & (df_gt_resol['x'] >= RESOLUCAO_ESP)) | (df_gt_resol['x'] == 0)) & 
            (((df_gt_resol['y'] <= LINEAR_MAX) & (df_gt_resol['y'] >= RESOLUCAO_ESP)) | (df_gt_resol['y'] == 0)) & 
            (((df_gt_resol['z'] <= LINEAR_MAX) & (df_gt_resol['z'] >= RESOLUCAO_ESP)) | (df_gt_resol['z'] == 0))]
        data_loss_percentage = 100 - len(df_gt_resol)/len(df) * 100
        if data_loss_percentage < best_percentage:
            best_percentage = data_loss_percentage
            best_gain = gain
    
    return best_percentage, best_gain

best_performance(df)

(1.0522895598917472, 52)